<a href="https://colab.research.google.com/github/vineboneto/ml/blob/main/thread.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ativdade 1

In [ ]:
from threading import Thread

NUM_THREADS = 4
PARAMS = [(1, 250000), (250001, 500000), (500001, 750000), (750001, 1000000)]

def task_sum(start, end, result):
    total = sum(range(start, end + 1))
    result.append(total)

if __name__ == '__main__':
    result = []
    threads = []
    
    for i in range(NUM_THREADS):
        thread = Thread(target=task_sum, args=(*PARAMS[i], result))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    print(f'total: {sum(result)}')



total: 500000500000


## Problemas dos filósofos

In [ ]:
from threading import Thread, Lock
from time import sleep
from random import randint



class Display:

    def __init__(self, nomes):
        self.state = [f"{nome}:P" for nome in nomes]
        self.lock = Lock()

    def display(self, nome, action, next_action):
        temp = self.state
        for count, state in enumerate(self.state):
            if state == f"{nome}:{action}":
                temp[count] = f"{nome}:{next_action}"

        self.state = temp
        print(f"{nome} -> " + " ".join(self.state))

class Filosofo(Thread):
    pensando = True

    def __init__(self, **kwargs):
        Thread.__init__(self)
        self.display = kwargs.get("display")
        self.nome = kwargs.get("nome")
        self.tempo_comendo = kwargs.get("tempo_comendo")
        self.tempo_pensando = kwargs.get("tempo_pensando")
        self.garfo_esquerda = kwargs.get("garfo_esquerda")
        self.garfo_direita = kwargs.get("garfo_direita")

    def run(self):
        n_sem_comer = 0
        for _ in range(100):
            # print(f"{self.nome} esta pensando por {self.tempo_pensando} segundos")
            sleep(self.tempo_pensando)
            conseguiu_comer = self.comer()

            if conseguiu_comer:
                n_sem_comer = 0
            else:
                n_sem_comer += 1
            
            # Starvation
            if n_sem_comer > 3:
                print(f"{self.nome} -> Não conseguiu comer por {n_sem_comer}")
                t = self.garfo_direita.acquire()
                e = self.garfo_esquerda.acquire()
                self.display.display(self.nome, "P", "C")
                sleep(self.tempo_comendo)
                self.display.display(self.nome, "C", "P")
                self.garfo_direita.release()
                self.garfo_esquerda.release()
                n_sem_comer = 0

    def comer(self):
        garfo1, garfo2 = self.garfo_esquerda, self.garfo_direita

        # Tenta pegar o primeiro garfo
        garfo1.acquire(True)

        # Se estiver livre retorna True e bloqueia
        # Se estiver bloqueado retorna False
        esta_livre = garfo2.acquire(False)

        if esta_livre:
            # Tempo comendo
            self.display.display(self.nome, "P", "C")
            # print(f"{self.nome} comecou a comer por {self.tempo_comendo} segundos")
            sleep(self.tempo_comendo)
            self.display.display(self.nome, "C", "P")
            # print(f"{self.nome} parou de comer e largou os garfos")
            garfo1.release()
            garfo2.release()
            return True
        else:
            # Solta o garfo
            print(f"{self.nome} -> Tentou e não conseguiu comer")
            garfo1.release()
            return False


if __name__ == "__main__":

    nomes = ("Arthur", "Gabriel", "Vitor", "Neymar", "Dijkstra")

    display = Display(nomes)

    garfos = [Lock() for _ in range(5)]

    mesa = [
        Filosofo(
            display=display,
            nome=nomes[i],
            tempo_pensando=randint(1, 20),
            tempo_comendo=randint(1, 20),
            garfo_esquerda=garfos[i % 5],
            garfo_direita=garfos[(i + 1) % 5],
        )
        for i in range(5)
    ]

    for filosofo in mesa:
        filosofo.start()
        sleep(1)


